# YOLO Train

## Read data

In [1]:
import os
import numpy as np
from read_tfrecord import get_dataset
from utils import read_anchors

%load_ext autoreload
%autoreload 2

In [3]:
train_path = './data/train_tiny.record'
test_path = './data/test_tiny.record'
batch_size = 8
train_data = get_dataset(train_path, [416, 416, 3], class_num=2, anchor_num=3, layer_num=2)
val_data = get_dataset(test_path, [416, 416, 3], class_num=2, anchor_num=3, layer_num=2)
batch = train_data.shuffle(10000).batch(batch_size)
val_batch = val_data.batch(batch_size)

### Read anchors

In [4]:
anchors = read_anchors('model/tiny_pet_anchors.txt')

## Build new model

In [5]:
from yolo_model import YoloV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [6]:
yolov3 = YoloV3(input_shape=(416, 416, 3), 
        num_classes=2,
        anchors=anchors,
        training=True,
        tiny=True
        )

In [8]:
yolov3.model.load_weights('model/tiny_yolo_base.h5', by_name=True)

for layer in yolov3.model.layers[:-2]:
    layer.trainable = False
    
yolov3.model.compile(Adam(1e-2),
        loss=[yolov3.yolo_loss(i) for i in range(len(yolov3.anchor_mask))])

yolov3.model.summary()

W0711 16:46:32.659005  3236 deprecation.py:323] From D:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\ops\array_ops.py:1340: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 416, 416, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 416, 416, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 416, 416, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

## Train

In [9]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min'),
             ModelCheckpoint(filepath='model/tiny_yolo_train.h5', monitor='val_loss', verbose=0, save_best_only=True),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)]

In [13]:
yolov3.model.fit(batch, epochs=5, callbacks=callbacks, validation_data=val_batch)

Epoch 1/5
415/415 [==============================] - 317s 764ms/step - loss: 30.3228 - conv2d_9_loss: 22.4781 - conv2d_12_loss: 3.6610 - val_loss: 0.0000e+00 - val_conv2d_9_loss: 0.0000e+00 - val_conv2d_12_loss: 0.0000e+00
Epoch 2/5
415/415 [==============================] - 315s 759ms/step - loss: 32.7952 - conv2d_9_loss: 24.9502 - conv2d_12_loss: 3.6296 - val_loss: 19.3861 - val_conv2d_9_loss: 11.5577 - val_conv2d_12_loss: 3.6066
Epoch 3/5
415/415 [==============================] - 310s 746ms/step - loss: 22.1559 - conv2d_9_loss: 14.0362 - conv2d_12_loss: 3.8866 - val_loss: 16.7882 - val_conv2d_9_loss: 8.9073 - val_conv2d_12_loss: 3.6395
Epoch 4/5
415/415 [==============================] - 307s 740ms/step - loss: 21.6508 - conv2d_9_loss: 13.7063 - conv2d_12_loss: 3.6872 - val_loss: 18.6748 - val_conv2d_9_loss: 10.8631 - val_conv2d_12_loss: 3.5426
Epoch 5/5
415/415 [==============================] - 312s 752ms/step - loss: 23.6175 - conv2d_9_loss: 15.3854 - conv2d_12_loss: 3.9496 - va

In [19]:
yolov3.model.save_weights('model/tiny_yolo_train.h5')